In [ ]:
!pip install transformers


In [ ]:
!pip install sentencepiece


In [ ]:
from transformers import AutoConfig, AutoTokenizer,AutoModelForSeq2SeqLM

model_name = "t5-small"
# config = AutoConfig.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# seq2seq = AutoModelForSeq2SeqLM.from_pretrained(model_name)

from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(model_name, use_fast=True, legacy=False)
#model = T5ForConditionalGeneration.from_pretrained(model_name, config=config)


config = AutoConfig.from_pretrained(
    model_name
  )
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
  )
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config
  )

In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset
dataset = load_dataset('findnitai/english-to-hinglish')


In [ ]:
train_dataset = dataset['train']
print(train_dataset[:5])

{'translation': [{'en': "What's the name of the movie", 'hi_ng': 'film ka kya naam hai', 'source': 1}, {'en': 'Hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. ', 'hi_ng': 'namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.', 'source': 1}, {'en': 'Do you think you will like the movie', 'hi_ng': 'kya aapako lagata hai ki aapako film pasand aaegee', 'source': 1}, {'en': 'What kind of movie is it', 'hi_ng': 'yah kis tarah kee philm hai', 'source': 1}, {'en': 'when was the movie made?', 'hi_ng': 'film  kab banee thee?', 'source': 1}]}


In [ ]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast
tokenizer_base = AutoTokenizer.from_pretrained("bert-base-cased")

master = []
for line in dataset['train']['translation']:
    master.append(line['en'])
    master.append(line['hi_ng'])

def gen_training_data():
    return (master[i : i+500]
    for i in range(0, len(master), 500)
    )
tokenizer_training_data = gen_training_data()
tokenizer = tokenizer_base.train_new_from_iterator(tokenizer_training_data, 32128)
tokenizer.save_pretrained("en-hig-tokenizer")

('en-hig-tokenizer/tokenizer_config.json',
 'en-hig-tokenizer/special_tokens_map.json',
 'en-hig-tokenizer/vocab.txt',
 'en-hig-tokenizer/added_tokens.json',
 'en-hig-tokenizer/tokenizer.json')

In [ ]:
# Use ! to execute the curl command in a Colab cell
!curl -X GET "https://datasets-server.huggingface.co/rows?dataset=findnitai%2Fenglish-to-hinglish&config=default&split=train&offset=0&limit=10000" -o dataset.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17489  100 17489    0     0  29639      0 --:--:-- --:--:-- --:--:-- 29642


In [ ]:
train_file = "hinglish_upload_v1.json"

# Load the dataset from the JSON file
data_files = {}
data_files["train"] = train_file
raw_datasets = load_dataset(
    "json",
    data_files=data_files
)
source_prefix = "Translate English to Hinglish : "
source_lang = "en"
target_lang = "hi_ng"
max_source_length = 128
max_target_length = 128 # target and source length task dependent (translation, summary etc.)
padding = "max_length" # padding to max length
num_epochs = 3

In [ ]:
def preprocess(source_data):
  inputs = [sample[source_lang] for sample in source_data["translation"]]
  targets = [sample[target_lang] for sample in source_data["translation"]]
  inputs = [source_prefix + inp for inp in inputs]
  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
  # Tokenize targets with the `text_target` keyword argument
  labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)
  # replace tokenizer.pad_token_id in the labels by -100 to ignore padding in the loss.
  labels["input_ids"] = [
      [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
from transformers import DataCollatorForSeq2Seq, T5Tokenizer

In [ ]:


# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,     # Set padding to True for batch collation
    max_length=128,   # Specify the maximum sequence length
    #max_padding=True
    #ecify the maximum target sequence length
    # truncation=True,  # Enable truncation of sequences if they exceed max_length
)

# Now, you can use data_collator to collate your training data batches


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(preprocess, batched=True, remove_columns="translation")
# data_collator = default_data_collator


Map:   0%|          | 0/189102 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 189102
    })
})

In [ ]:
data_collator = data_collator

In [ ]:
trainer_args_in = {
    'output_dir': 'my-t5-hinglish-translator',
    'overwrite_output_dir': True,
    'do_train': True,
    'per_device_train_batch_size': 8,
    'num_train_epochs': 1,     # Set the number of epochs to 3
    #'max_steps': 15000,        # Set the maximum number of training steps to 15,000
}

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]


In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html



Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.

In [ ]:
!pip install transformers[torch] accelerate

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
from transformers import HfArgumentParser, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Assuming you have already defined your model, tokenizer, train_dataset, and data_collator
# ...
training_args = Seq2SeqTrainingArguments(
    output_dir="my-t5-hinglish-translator",
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    num_train_epochs=num_epochs,
    save_total_limit=1,
    save_steps=2,  # Adjust as needed
    logging_steps=2,  # Adjust as needed
    #max_steps=20000
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)
parser = HfArgumentParser((Seq2SeqTrainingArguments))
training_args = parser.parse_dict(trainer_args_in)

trainer = Seq2SeqTrainer(model=model, args=training_args[0], train_dataset=train_dataset, tokenizer=tokenizer, data_collator=data_collator)

train_result = trainer.train(resume_from_checkpoint=None)
trainer.save_model()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
500,3.055700
1000,2.256000
1500,1.993700
2000,1.815000
2500,1.648800
3000,1.585100
3500,1.511600
4000,1.435100
4500,1.369400
5000,1.340300


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")
input_ids = tokenizer("translate English to Hinglish:waether is warm", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test Output : hua hua hua hua hu
